In [42]:
import numpy as np
import random
import pickle
import time

In [43]:
learning_rate=0.1 #alpha
decay_gamma=0.7 #gamma
exp_rate=0.3 #epsilon
starting_player='agent'
list_of_winners=[]
policy={}
number_of_pits=2
number_of_stones=2

In [134]:
class impartial_kalaha:
    def __init__(self, pits, pebbles):
        self.starting_player=starting_player
        self.player=starting_player
        self.pits=pits
        self.pebbles=pebbles
        self.board=[pebbles]*pits+[0]
        self.number_of_pits=len(self.board)
        self.lr=learning_rate
        self.gamma=decay_gamma
        self.exp_rate=exp_rate
        self.Q-table={}
        self.game_over=False
        self.states=[]
    def choose_random_pit(self):
        """This function chooses a random pit to choose.
        Returns an index for a non-empty game pit"""
        return random.choice(list(np.nonzero(self.board[:-1])[0]))
    def choose_smart_pit(self):
        """This function chooses the most optimal non-empty game pit according to the Q-table.
        Returns the index for the best choice of non-empty game pit"""
        current_board=self.board.copy()
        boards_to_evaluate_in_Q_table=[]
        possible_moves=list(np.nonzero(current_board[:-1])[0])
        for pit_to_choose in possible_moves:
            test_board=current_board.copy()
            _, __, test_board = self.test_move(pit_to_choose, test_board)
            boards_to_evaluate_in_Q_table.append((pit_to_choose,test_board))
        value_max=-999
        best_index=None
        for next_board in boards_to_evaluate_in_Q_table:
            value = 0 if self.Q-table.get(str(next_board[1])) is None else self.Q-table.get(str(next_board[1]))
            if value >= value_max:
                value_max = value
                best_index=next_board[0]
        return best_index
    def possible_next_states(self, init_board):
        """This function finds all possible board states achieveable from a given state.
        Input is a board for which to find achieveable board states from
        Returns a list of board states achieveable from the input board."""
        list_of_board_states=[]
        boards_to_check=[init_board.copy()]
        while len(boards_to_check)>0:
            current_board=boards_to_check.pop(0)
            possible_moves=list(np.nonzero(current_board[:-1])[0])
            for pit_to_choose in possible_moves:
                test_board=current_board.copy()
                turn='ongoing'
                game_over=False
                turn, game_over, test_board = self.test_move(pit_to_choose, test_board)
                if turn=='over' or game_over==True:
                    list_of_board_states.append(test_board)
                else:
                    boards_to_check.append(test_board)
        return [list(x) for x in set(tuple(subliste) for subliste in list_of_board_states)]     
    def test_move(self, pit_index, test_board):
        """This function is for testing the outcome of choosing a certain non-empty game pit from a certain game state
        Input is the index of the non-empty game pit and the board state to test from.
        Returns:
            turn: is either ongoing or over, depending on whether the move ends the turn or requires another move
            game_over: is true if the move results in all pebbles being in the target pit.
            test_board: the board state after the move is done"""
        turn='ongoing'
        pit_to_choose=pit_index
        game_over=False
        while turn=='ongoing' and game_over==False:
            pebbles_on_hand=test_board[pit_to_choose]
            test_board[pit_to_choose]=0
            for i in range(pebbles_on_hand):
                pit_to_add_to=(pit_to_choose+1+i)%(len(test_board))
                test_board[pit_to_add_to]+=1
            if pit_to_add_to==len(test_board)-1: #if last stone is placed in j0
                if test_board[-1]==sum(test_board): #if it is the last stone in the game
                    game_over=True
                    turn='over'
                else:
                    return turn, game_over, test_board
            elif test_board[pit_to_add_to]==1:#if last stone is placed in empty pit
                turn='over'
            else:
                pit_to_choose=pit_to_add_to #last stone is not placed in empty pit, so continue
        return turn, game_over, test_board
    def perform_move(self, pit_index):
        """Function to perform a move to the actual game board.
        input: pit_index is the index(0-indexed) of the pit in the board which the player
                should take the pebbles from
        Returns whether or not the turn is over by performing the move."""
        turn='ongoing'
        pit_to_choose=pit_index
        while turn=='ongoing' and self.game_over==False:
            pebbles_on_hand=self.board[pit_to_choose]
            self.board[pit_to_choose]=0
            for i in range(pebbles_on_hand):
                pit_to_add_to=(pit_to_choose+1+i)%self.number_of_pits
                self.board[pit_to_add_to]+=1
            if pit_to_add_to==len(self.board)-1: #if last stone is placed in j0
                if self.board[-1]==self.pebbles*self.pits: #if it is the last stone in the game
                    self.game_over=True
                    turn='over'
                else:
                    return turn
            elif self.board[pit_to_add_to]==1:#if last stone is placed in empty pit
                turn='over'
            else:
                pit_to_choose=pit_to_add_to #last stone is not placed in empty pit, so continue
        return turn
    def change_player(self):
        """Function for changing current player.
        Returns None"""
        if self.player=='agent':
            self.player='dummy'
        else:
            self.player='agent'
    def reset(self):
        """Function for resetting the game.
        Returns None"""
        self.game_over=False
        winner=None
        self.player=starting_player
        self.j0=0
        self.board=[self.pebbles]*self.pits+[0]
        self.states=[]
    def feedReward(self, reward):
        """Function for updating the Q-table.
        Input is the reward/penalty to provide board states from a game.
        Returns None"""
        for st in reversed(self.states):  # goes through all saved board states of this game
            temp=-1000
            possible_next_st=self.possible_next_states(st)
            for i in possible_next_st:
                if self.Q-table.get(str(i)) is not None and self.Q-table.get(str(i))>temp:
                    temp=self.Q-table.get(str(i))
            if self.Q-table.get(str(st)) is None:
                self.Q-table[str(st)] = 0  # initialise a value for the state
            self.Q-table[str(st)] += self.lr * (reward -self.gamma*temp- self.Q-table[str(st)])
    def training_game(self, rounds=1000):
        """Function for training the agent.
        Returns None"""
        for i in range(rounds):
            while not self.game_over:
                if self.player=='dummy':
                    turn='ongoing'
                    while turn=='ongoing':
                        move= self.choose_random_pit()
                        turn=self.perform_move(move)
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        turn='ongoing'
                        while turn=='ongoing':
                            move= self.choose_random_pit()
                            turn=self.perform_move(move)
                            self.states.append(self.board)
                    else:
                        turn='ongoing'
                        while turn=='ongoing':
                            move= self.choose_smart_pit()
                            turn=self.perform_move(move)
                            self.states.append(self.board)
                if self.game_over:
                    winner=self.player
                    list_of_winners.append(winner)
                    if winner=='agent':
                        self.feedReward(5)
                    else:
                        self.feedReward(-1)
                    self.reset()
                    break
                else:
                    self.change_player()
    def testing_game(self, rounds=1000):
        """Function for testing the agent using the Q-table obtained from training.
        Returns None"""
        for i in range(rounds):
            while not self.game_over:
                if self.player=='dummy':
                    turn='ongoing'
                    while turn=='ongoing':
                        move= self.choose_random_pit()
                        turn=self.perform_move(move)
                else:
                        turn='ongoing'
                        while turn=='ongoing':
                            move= self.choose_smart_pit()
                            turn=self.perform_move(move)
                            self.states.append(self.board)
                if self.game_over:
                    winner=self.player
                    list_of_winners.append(winner)
                    self.reset()
                    break
                else:
                    self.change_player()

In [135]:
hej=impartial_kalaha(2,4)

In [136]:
list_of_winners=[]
hej.training_game(1000)

In [137]:
list_of_winners.count('agent')

929

In [138]:
list_of_winners=[]
hej.testing_game(1000)

In [139]:
list_of_winners.count('agent')

1000

In [18]:
hej.perform_move(1)

'over'

In [105]:
hej.perform_move(1)

halløj [0, 2, 4, 1, 1]
halløj [0, 0, 5, 2, 1] end
halløj [1, 0, 5, 0, 2] end


In [106]:
hej.perform_move(0)

halløj [1, 0, 5, 0, 2]
halløj [0, 1, 5, 0, 2] end


In [107]:
hej.perform_move(1)

halløj [0, 1, 5, 0, 2]
halløj [0, 0, 6, 0, 2] end
halløj [1, 1, 1, 2, 3] end
halløj [2, 1, 1, 0, 4] end
halløj [0, 2, 2, 0, 4] end
halløj [0, 2, 0, 1, 5] end
halløj [0, 0, 1, 2, 5] end
halløj [1, 0, 1, 0, 6] end


In [109]:
hej.perform_move(1)

halløj [0, 1, 1, 0, 6]
halløj [0, 0, 2, 0, 6] end
halløj [0, 0, 0, 1, 7] end
GAME OVER
halløj [0, 0, 0, 0, 8] end
